In [1]:
import requests
from bs4 import BeautifulSoup
# import lxml.html as lh
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
import time
import json
from tqdm import tqdm
from sqlalchemy import create_engine

from bokeh.models.glyphs import Arc
from math import pi
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Reds6,Blues8,RdYlBu,Reds,Inferno256

In [2]:
from bokeh.io import reset_output
import numpy as np

from bokeh.io import output_file, show, reset_output
from bokeh.plotting import figure
from bokeh.transform import linear_cmap
from bokeh.models import ColorBar,LinearColorMapper,LogColorMapper,ContinuousColorMapper
from bokeh.util.hex import hexbin
from bokeh.palettes import RdBu,Oranges,Turbo256,RdPu
from bokeh.models import HoverTool
from bokeh.io.export import export_png

In [4]:
from Scraper import get_schedule

# Drawing

In [5]:
def get_games_events(teamid=None,scheduleid_list=None):
    """
    从database中，根据teamid或scheduleid，获取比赛的所有events，包括对手的events。
    注意，即使值提供teamid参数，也会获取对手的events。
    
    """
    assert (not (teamid is None)&(scheduleid_list is None)),'至少需要teamid或scheduleid_list一个参数'
    user_name = 'master'
    passcode = 'Pw#cbashuju0131'
    endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
    database = 'CBA_Data'
    #     database = 'CBA_Staging'
    engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
    connection= engine.connect()
    if scheduleid_list:
        assert type(scheduleid_list)==list,'scheduleid_list只接受list'
        sql_str = f"""
        SELECT *
        FROM CBA_Data.Playbyplay plays1
        WHERE plays1.scheduleid IN ('{"','".join(str(i) for i in scheduleid_list)}')
        """
    else:
        sql_str = f"""
        SELECT *
        FROM CBA_Data.Playbyplay plays1
        WHERE plays1.scheduleid IN (
            SELECT DISTINCT plays2.scheduleid
            FROM CBA_Data.Playbyplay plays2
            WHERE plays2.teamid = {teamid}
        )
        """
    team_events = pd.read_sql(sql_str, connection)
    # plays.to_sql(name='Playbyplay',con=connection,index=False,if_exists='replace')
    connection.close()
    return team_events

In [6]:
def get_shotevents(games_events,teamid=None,player_name=None):
    """
    此函数能够根据输入的playbyplay数据，筛选出投篮事件。
    若指定某队teamid，则可以筛选特定某队的投篮事件。
    若指定某队teamid和球员名字，则可以筛选特定某队员的投篮事件。
    """
    games_shotevents = games_events.loc[pd.notna(games_events['shotevent']),].copy()

    games_shotevents['shotcoordx'] = games_shotevents['shotcoordx'].astype('float')
    games_shotevents['shotcoordy'] = games_shotevents['shotcoordy'].astype('float')
    
    if teamid:
        games_shotevents = games_shotevents.loc[games_shotevents['teamid']==teamid]
        if player_name:
            games_shotevents = games_shotevents.loc[(games_shotevents['event'].str.contains('player_name'))&(-games_shotevents['event'].str.contains(player_name+'抢到'))&(-(games_shotevents['event'].str.contains(player_name+'的助攻'))),:]
            
    
    
    return games_shotevents

### 球队投篮分布

In [7]:
import math
from math import pi

In [8]:
def meter_to_coord(meter):
    return meter*49/15

In [9]:
def angle(vector1, vector2):
    x1, y1 = vector1
    x2, y2 = vector2
    inner_product = x1*x2 + y1*y2
    len1 = math.hypot(x1, y1)
    len2 = math.hypot(x2, y2)
    return math.acos(inner_product/(len1*len2))

In [10]:
def draw_court(p):
    endline_y = -7.5
    halfcourtline_y=45.73
    sideline_left = 24.5
    rim_to_baseline=7.5
    rim_x=0
    rim_y=endline_y+meter_to_coord(1.2)
    threepointline_radius = meter_to_coord(6.75)
    nochargeline_rarius = meter_to_coord(1.25)
    centercircle_radius = meter_to_coord(1.83)
    rim_radius = meter_to_coord(0.23)
    backboard_length = meter_to_coord(1.83)
    restrictedarea_length = meter_to_coord(5.8)
    restrictedarea_width = meter_to_coord(4.9)
    ftline_length = meter_to_coord(3.6)


    goal_color = Blues8[1]
    miss_color = Reds6[0]
    linewidth = 3
    size = 9
    alpha = 0.6
    
    # 3p line
    p.arc(x=rim_x, y=rim_y, radius=threepointline_radius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 左侧三分直线
    p.line(x=[-threepointline_radius,-threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 右侧三分直线
    p.line(x=[threepointline_radius,threepointline_radius],y=[endline_y,rim_y],line_color='black', line_width=linewidth-0.1)

    # 合理冲撞区
    p.arc(x=rim_x, y=rim_y, radius=nochargeline_rarius, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)

    # 中圈
    p.arc(x=rim_x, y=halfcourtline_y, radius=centercircle_radius, start_angle=pi, end_angle=0,line_color='black', line_width=linewidth)

    # 篮筐
    p.circle(x=0,y=rim_y,radius = rim_radius,color='red',fill_color=None,line_width=linewidth)


    # 篮板
    p.line(x=[-backboard_length/2,backboard_length/2],
           y=[rim_y-rim_radius-0.3,rim_y-rim_radius-0.3],
           line_color='black', line_width=linewidth)

    #场地边线
    p.quad(-sideline_left,sideline_left,halfcourtline_y,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #限制区
    p.quad(-restrictedarea_width/2,restrictedarea_width/2,endline_y+restrictedarea_length,endline_y,fill_color=None,line_color='black',
          line_width=linewidth)

    #罚球区
    p.arc(x=rim_x, y=endline_y+restrictedarea_length, radius=ftline_length/2, start_angle=0, end_angle=pi,line_color='black', line_width=linewidth)


In [29]:
endline_y = -7.5
halfcourtline_y=45.73
sideline_left = 24.5
rim_to_baseline=7.5
rim_x=0
rim_y=endline_y+meter_to_coord(1.2)
threepointline_radius = meter_to_coord(6.75)
nochargeline_rarius = meter_to_coord(1.25)
centercircle_radius = meter_to_coord(1.83)
rim_radius = meter_to_coord(0.23)
backboard_length = meter_to_coord(1.83)
restrictedarea_length = meter_to_coord(5.8)
restrictedarea_width = meter_to_coord(4.9)
ftline_length = meter_to_coord(3.6)
goal_color = Blues8[1]
miss_color = Reds6[0]
linewidth = 3
size = 9
alpha = 0.6

def new_figure(plot_title='标题'):
    # create a new plot
    reset_output()
    output_notebook()

    p = figure(
       title=plot_title,tools="save,reset",
        x_range=[-sideline_left,sideline_left],y_range = [endline_y,halfcourtline_y],x_axis_label='x',y_axis_label='y',
        match_aspect=True,background_fill_color='white'
    )
    
    p.axis.visible = False
    p.grid.visible = False
    
    return p

    
def plot_shot_dist_dot(p,df_shotevents,color_miss = Reds6[0], color_goal=Blues8[1],size=9,alpha=0.6):


    # misses
    p.cross(df_shotevents.loc[df_shotevents['eventtypeid']=='4','shotcoordx'],
            df_shotevents.loc[df_shotevents['eventtypeid']=='4','shotcoordy'],
            angle=90,color=color_miss, size=size,alpha=alpha,legend_label='未命中')

    # goals
    p.circle(df_shotevents.loc[df_shotevents['eventtypeid']=='3','shotcoordx'], 
             df_shotevents.loc[df_shotevents['eventtypeid']=='3','shotcoordy'], 
             fill_color=color_goal,color=color_goal, size=size,alpha=alpha,legend_label='命中')
    

    #width: 15m, 50
    #length: 28m，

def plot_shot_dist_hex(p,df_shotevents,palette = Inferno256,
                       hex_size=2,dot_size=5,thres_low=10,thres_high=300,plot_color_bar=False,
                      alpha=0.8):

    x = df_shotevents['shotcoordx'].values
    y = df_shotevents['shotcoordy'].values
    
    color_grade = palette
    
    bins = hexbin(x, y, hex_size)
    
    p.hex_tile(q="q", r="r", size=hex_size, line_color=None, source=bins, 
               fill_color=linear_cmap('counts', color_grade, low=thres_low, 
                                      high=max(bins.counts),low_color='white')
               ,alpha=alpha)
    
    # 因为2分和3分
    if plot_color_bar:
        color_mapper = LinearColorMapper(palette=palette, low=thres_low, high=thres_high,low_color='white')
        color_bar = ColorBar(title='出手频率',color_mapper=color_mapper, label_standoff=5, 
                             border_line_color=None, location=(0,0),width=20,
                             major_label_overrides={50:'低',100:'',150:'',200:'',250:'',300:'高'},
                             title_standoff=10)
        p.add_layout(color_bar, 'right')
#     return p

# 分析

### 本赛季两队投篮分布

In [12]:
liaoning = '29129'
guangdong = '29124'

In [15]:
teamid=liaoning
games_events = get_games_events(teamid)

# 去除总决赛第二场的影响
games_events=games_events.drop(index=games_events.query("scheduleid=='100042218'").index)

liaoning_shotevents = get_shotevents(games_events,teamid=liaoning)
liaoning_shotevents_3pt = liaoning_shotevents.loc[liaoning_shotevents['event'].str.rfind('三分')!=-1]
liaoning_shotevents_2pt = liaoning_shotevents.loc[liaoning_shotevents['event'].str.rfind('三分')==-1]

p=new_figure('辽宁赛季投篮分布图')
# 分别画2分和3分的distribution，以防3分出手数被内线出手数埋没，全都显示地出手频率
# 两个plot中只需画一个colorbar
plot_shot_dist_hex(p,liaoning_shotevents_3pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300)
plot_shot_dist_hex(p,liaoning_shotevents_2pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,plot_color_bar=True)
draw_court(p)
show(p)

Loading BokehJS ...

In [17]:
teamid=guangdong
games_events = get_games_events(teamid)


# 去除总决赛第二场的影响
games_events=games_events.drop(index=games_events.query("scheduleid=='100042218'").index)

guangdong_shotevents = get_shotevents(games_events,teamid=teamid)
guangdong_shotevents_3pt = guangdong_shotevents.loc[guangdong_shotevents['event'].str.rfind('三分')!=-1]
guangdong_shotevents_2pt = guangdong_shotevents.loc[guangdong_shotevents['event'].str.rfind('三分')==-1]

# 把figure分离出来，可覆盖多重plot
p=new_figure('广东赛季投篮分布图')
plot_shot_dist_hex(p,guangdong_shotevents_3pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300)
plot_shot_dist_hex(p,guangdong_shotevents_2pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,plot_color_bar=True)
draw_court(p)
show(p)

Loading BokehJS ...

# 总决赛第二场两队单场投篮散点图

In [22]:
games_events = get_games_events(liaoning,['100042218'])

liaoning_shotevents = get_shotevents(games_events,teamid=liaoning)

p=new_figure('总决赛第二场辽宁投篮分布图')
plot_shot_dist_dot(p,liaoning_shotevents)
draw_court(p)
show(p)

Loading BokehJS ...

In [23]:
games_events = get_games_events(guangdong,['100042218'])

guangdong_shotevents = get_shotevents(games_events,teamid=guangdong)

p=new_figure('总决赛第二场广东投篮分布图')
plot_shot_dist_dot(p,guangdong_shotevents)
draw_court(p)
show(p)

Loading BokehJS ...

### 两队赛季区域图和单场散点图相结合

In [30]:
teamid=liaoning
team_season_events = get_games_events(teamid)

# 去除总决赛第二场的影响
team_season_events=team_season_events.drop(index=team_season_events.query("scheduleid=='100042218'").index)
team_season_shotevents = get_shotevents(team_season_events,teamid=teamid)

team_season_shotevents_3pt = team_season_shotevents.loc[team_season_shotevents['event'].str.rfind('三分')!=-1]
team_season_shotevents_2pt = team_season_shotevents.loc[team_season_shotevents['event'].str.rfind('三分')==-1]

# 单场
team_game_events = get_games_events(teamid,['100042218'])
team_game_shotevents = get_shotevents(team_game_events,teamid=teamid)


p=new_figure('辽宁投篮分布图：赛季VS第二场')
plot_shot_dist_hex(p,team_season_shotevents_3pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,alpha=0.6)
plot_shot_dist_hex(p,team_season_shotevents_2pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,plot_color_bar=True,alpha=0.6)
plot_shot_dist_dot(p,team_game_shotevents,size=15,alpha=1)
draw_court(p)
show(p)

Loading BokehJS ...

In [31]:
teamid=guangdong
team_season_events = get_games_events(teamid)

# 去除总决赛第二场的影响
team_season_events=team_season_events.drop(index=team_season_events.query("scheduleid=='100042218'").index)
team_season_shotevents = get_shotevents(team_season_events,teamid=teamid)

team_season_shotevents_3pt = team_season_shotevents.loc[team_season_shotevents['event'].str.rfind('三分')!=-1]
team_season_shotevents_2pt = team_season_shotevents.loc[team_season_shotevents['event'].str.rfind('三分')==-1]

# 单场
team_game_events = get_games_events(teamid,['100042218'])
team_game_shotevents = get_shotevents(team_game_events,teamid=teamid)


p=new_figure('广东投篮分布图：赛季VS第二场')
plot_shot_dist_hex(p,team_season_shotevents_3pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,alpha=0.6)
plot_shot_dist_hex(p,team_season_shotevents_2pt,palette = RdBu[11],
                       hex_size=2,dot_size=5,thres_low=40,thres_high=300,plot_color_bar=True,alpha=0.6)
plot_shot_dist_dot(p,team_game_shotevents,size=15,alpha=1)
draw_court(p)
show(p)

Loading BokehJS ...

# 两队总决赛两场出手对比

In [55]:
game1_events = get_games_events(liaoning,['100042217'])
game1_shotevents = get_shotevents(game1_events,teamid=liaoning)

team_game1_shotevents_3pt = game1_shotevents.loc[game1_shotevents['event'].str.rfind('三分')!=-1]
team_game1_shotevents_2pt = game1_shotevents.loc[game1_shotevents['event'].str.rfind('三分')==-1]


game2_events = get_games_events(liaoning,['100042218'])
game2_shotevents = get_shotevents(game2_events,teamid=liaoning)

p=new_figure('辽宁总决赛第一场VS第二场')

plot_shot_dist_hex(p,team_game1_shotevents_3pt,palette = RdBu[11],
                       hex_size=1,dot_size=5,thres_low=1,thres_high=5,alpha=0.6)
plot_shot_dist_hex(p,team_game1_shotevents_2pt,palette = RdBu[11],
                       hex_size=1,dot_size=5,thres_low=1,thres_high=20,alpha=0.6)
plot_shot_dist_dot(p,game2_shotevents,alpha=1)

draw_court(p)
show(p)

Loading BokehJS ...

In [56]:
game1_events = get_games_events(guangdong,['100042217'])
game1_shotevents = get_shotevents(game1_events,teamid=guangdong)

team_game1_shotevents_3pt = game1_shotevents.loc[game1_shotevents['event'].str.rfind('三分')!=-1]
team_game1_shotevents_2pt = game1_shotevents.loc[game1_shotevents['event'].str.rfind('三分')==-1]


game2_events = get_games_events(guangdong,['100042218'])
game2_shotevents = get_shotevents(game2_events,teamid=guangdong)

p=new_figure('广东总决赛第一场VS第二场')

plot_shot_dist_hex(p,team_game1_shotevents_3pt,palette = RdBu[11],
                       hex_size=1,dot_size=5,thres_low=1,thres_high=5,alpha=0.6)
plot_shot_dist_hex(p,team_game1_shotevents_2pt,palette = RdBu[11],
                       hex_size=1,dot_size=5,thres_low=1,thres_high=20,alpha=0.6)
plot_shot_dist_dot(p,game2_shotevents,size=15,alpha=1)

draw_court(p)
show(p)

Loading BokehJS ...

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### ignore below

### 周琦vs易建联

In [227]:
zq_shots = xj_shots.loc[(xj_shots['event'].str.contains('周琦'))&(-xj_shots['event'].str.contains('周琦抢到'))&(-(xj_shots['event'].str.contains('周琦的助攻'))),:].copy()


In [226]:
yjl_shots = gd_shots.loc[(gd_shots['event'].str.contains('易建联'))&(-gd_shots['event'].str.contains('易建联抢到'))&(-(gd_shots['event'].str.contains('易建联的助攻'))),:].copy()


In [241]:
print(f'易建联共{296+94}投{168+34}中，总命中率{(168+34)/(296+94)*100:.2f}%，3分投篮占比{94/(296+94)*100:.2f}%')

易建联共390投202中，总命中率51.79%，3分投篮占比24.10%


In [240]:
print(f'周琦共{243+103}投{33+157}中，总命中率{(33+157)/(243+103)*100:.2f}%，3分投篮占比{103/(243+103)*100:.2f}%')

周琦共346投190中，总命中率54.91%，3分投篮占比29.77%


In [379]:
from bokeh.models.glyphs import Arc
from math import pi

# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="周琦19-20赛季投篮分布--共346投190中，总命中率54.91%，3分投篮占比29.77%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='x',y_axis_label='y'
)

p.axis.visible = False

# misses
p.cross(zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordx'],zq_shots.loc[zq_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordx'], zq_shots.loc[zq_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...

In [380]:
# prepare some data

# output to static HTML file
output_notebook()

# create a new plot
p = figure(
   title="易建联19-20赛季投篮分布--共390投202中，总命中率51.79%，3分投篮占比24.10%",tools="pan,box_zoom,wheel_zoom,save,reset,hover",
    x_range=[-24.5,24.5],y_range = [-7.5,45.73],x_axis_label='xxx',y_axis_label='yyy'
)
p.axis.visible = False

# misses
p.cross(yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordx'],yjl_shots.loc[yjl_shots['eventtypeid']=='4','shotcoordy'],
        angle=90,color=miss_color, size=10,alpha=alpha,legend_label='未命中')

# goals
p.circle(yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordx'], yjl_shots.loc[yjl_shots['eventtypeid']=='3','shotcoordy'], 
         fill_color=goal_color,color=goal_color, size=10,alpha=alpha,legend_label='命中')

draw_court(p)

# show the results
show(p)

#width: 15m, 50
#length: 28m，


Loading BokehJS ...